In [2]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

/home/kuper/projects/datich/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 1. Define paths
base_model_id = "Qwen/Qwen2.5-0.5B"
adapter_path = "./qwen_lora" # Path to your unzipped downloaded folder

# 2. Load Tokenizer (saved along with your LoRA weights)
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

In [4]:
# 3. Load Base Model
# IMPORTANT: Use the exact same torch_dtype you used during training (bfloat16 or float16)
base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_id,
    num_labels=6,
    problem_type="regression",
    device_map="auto",
    torch_dtype=torch.bfloat16 
)
base_model.config.pad_token_id = tokenizer.pad_token_id

# 4. Snap the LoRA adapters onto the base model
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval() # Lock the model into evaluation mode

`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 290/290 [00:01<00:00, 258.86it/s, Materializing param=model.norm.weight]                              
Qwen2ForSequenceClassification LOAD REPORT from: Qwen/Qwen2.5-0.5B
Key          | Status  | 
-------------+---------+-
score.weight | MISSING | 

Notes:
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Qwen2ForSequenceClassification(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
             

In [5]:
# The order must match the target_columns from your training script exactly
emotions = [
    'sadness', 'anxiety', 'rumination', 'self_focus', 
    'hopelessness', 'emotional_volatility'
]

def analyze_text(text):
    # Tokenize the input text
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        max_length=512, 
        padding=True
    )
    
    # Move inputs to the same device (GPU/CPU) as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Run the forward pass without tracking gradients
    with torch.no_grad():
        outputs = model(**inputs)
        # Extract the 6 regression scores from the model's output
        raw_scores = outputs.logits[0].float().cpu().numpy()

    # Enforce the 0.0 to 1.0 bounds
    clipped_scores = np.clip(raw_scores, 0.0, 1.0)

    # Zip the emotion names with their scores into a clean JSON-like dictionary
    results = {
        emotion: round(float(score), 3) 
        for emotion, score in zip(emotions, clipped_scores)
    }
    
    return results

In [6]:
sample_text = "I've been pacing around my room for hours replaying that conversation. I just can't shake the feeling that I ruined everything and there's no way to fix it."

predictions = analyze_text(sample_text)
print(predictions)

# Expected Output shape: 
# {'sadness': 0.812, 'anxiety': 0.945, 'rumination': 0.88, 'self_focus': 0.72, 'hopelessness': 0.65, 'emotional_volatility': 0.41}

{'sadness': 0.656, 'anxiety': 0.236, 'rumination': 0.385, 'self_focus': 0.559, 'hopelessness': 0.535, 'emotional_volatility': 0.0}


In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 1. Load the scores your Qwen model generated for your dataset
df = pd.read_csv("qwen_generated_scores.csv")
features = ['sadness', 'anxiety', 'rumination', 'self_focus', 'hopelessness', 'emotional_volatility']
X = df[features]

# 2. Scale the data
# Standardization is mandatory for distance-based clustering algorithms
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Apply K-Means Clustering
# Let's assume we want to find 4 distinct profiles
kmeans = KMeans(n_clusters=4, random_state=42, n_init='auto')
df['cluster_id'] = kmeans.fit_predict(X_scaled)

# 4. Profile the Centroids
# This calculates the average score for each emotion within each cluster
cluster_centroids = df.groupby('cluster_id')[features].mean()
display(cluster_centroids)